In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ast
import urllib3
urllib3.disable_warnings()
header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

In [2]:
ürün_bilgileri=[]       
def açiklamalari_cek(model,soup2,ürün_marka,url,ürün_fiyat,ürün_adi,ürün_puan,ürün_resim):
    try:
        global işletim_sistemi,işlemci_nesli,ram,disk_kapasitesi,disk_türü,ekran_boyutu,işlemci_tipi   
        aciklamalar=soup2.find_all("table",attrs={"class":"product-table"})
        for x in aciklamalar:
            dolas=x.find_all("tr")
            for y in dolas:
                dolas2=y.find("td").get_text()
                if dolas2.find("İşletim Sistemi")!=-1:
                    işletim_sistemi=y.find("p").get_text().replace(" ","").upper().strip()
                    if işletim_sistemi.find("WINDOWS")!=-1:
                        işletim_sistemi="WINDOWS"
                    elif işletim_sistemi.find("YOK")!=-1:
                        işletim_sistemi="FREEDOS"
                    elif işletim_sistemi.find("DOS")!=-1:
                        işletim_sistemi="FREEDOS"
                    elif işletim_sistemi.find("CHROMEOS")!=-1:
                        işletim_sistemi="CHROMEOS"
                    elif işletim_sistemi.find("MACOS")!=-1:
                        işletim_sistemi="MACOS"
                elif dolas2.find("İşlemci Nesli")!=-1:
                    işlemci_nesli=y.find("p").get_text().strip()
                    if işlemci_nesli.find("Intel")!=-1:
                        işlemci_nesli=işlemci_nesli.replace("Intel","").strip().upper()
                    elif işlemci_nesli.find("AMD")!=-1:
                        işlemci_nesli=işlemci_nesli.replace("AMD","").strip().upper()
                    işlemci_nesli=işlemci_nesli.upper().strip()
                elif dolas2.find("Disk Türü")!=-1:
                    formatla=y.find("p").get_text().find("SSD")
                    formatla2=y.find("p").get_text().find("HDD")
                    if formatla!=-1 and formatla2!=-1:
                        disk_türü="SSD-HDD"
                    elif formatla2!=-1:
                        disk_türü="HDD"
                    else:
                        disk_türü="SSD"
                elif dolas2.find("Disk Kapasitesi")!=-1:
                    disk_kapasitesi=y.find("p").get_text().strip()
                elif dolas2.find("Ekran Boyutu")!=-1:
                    ekran_boyutu=y.find("p").get_text().strip()
                    if ekran_boyutu.find("inch")!=1:
                        ekran_boyutu=ekran_boyutu.replace("inch","")
                elif dolas2.find("İşlemci Teknolojisi")!=-1:
                    işlemci_tipi=y.find("p").get_text().upper().strip()
                    if işlemci_tipi.find("CORE")!=-1:
                        işlemci_tipi="INTEL "+işlemci_tipi
                    elif işlemci_tipi.find("RYZEN")!=-1:
                        işlemci_tipi="AMD "+işlemci_tipi
                    elif işlemci_tipi.find("CELERON")!=-1:
                        işlemci_tipi="INTEL CELERON"
                elif dolas2.find("Ram (Sistem Belleği")!=-1:
                    ram=y.find("p").get_text().strip()
                    if ram.find("GB")!=-1:
                        ram=ram[0:ram.find("GB")+2]
                    if(ram.find("TB")!=-1):
                        ram=ram[0:ram.find("TB")+2]
    except:
        pass
    else:
        print(ürün_marka,ürün_adi,model,işletim_sistemi,işlemci_tipi,işlemci_nesli,ram,disk_kapasitesi,disk_türü,ekran_boyutu,ürün_puan,ürün_resim,ürün_fiyat,url)
        ürün_bilgileri.append([ürün_marka,ürün_adi,model,işletim_sistemi,işlemci_tipi,işlemci_nesli,ram,disk_kapasitesi,disk_türü,ekran_boyutu,ürün_puan,ürün_resim,ürün_fiyat,url])

        
def ürün_bilgileri_bul(url):
    try:
        istek2=requests.get(url,headers=header)
        soup2=BeautifulSoup(istek2.content,"lxml")
        ürün_adi=soup2.find("h1",attrs={"class":"product-list__product-name"}).get_text()
        ürün_adi=ürün_adi.upper().strip()
        ürün_fiyat=soup2.find("span",attrs={"class":"product-list__price"}).get_text().strip()
        ürün_resim=soup2.find("img",attrs={"class":"swiper-lazy img-responsive wrapper-main-slider__image"}).get("data-srcset")
        puanlar=soup2.find_all("div",attrs={"class":"tab-pane--star-score"})
        puan5=float(puanlar[0].find("div",attrs={"class":"starTxt5"}).get_text().replace("%",""))*5.0
        puan4=float(puanlar[1].find("div",attrs={"class":"starTxt4"}).get_text().replace("%",""))*4.0
        puan3=float(puanlar[2].find("div",attrs={"class":"starTxt3"}).get_text().replace("%",""))*3.0
        puan2=float(puanlar[3].find("div",attrs={"class":"starTxt2"}).get_text().replace("%",""))*2.0
        puan1=float(puanlar[4].find("div",attrs={"class":"starTxt1"}).get_text().replace("%",""))*1.0
        ürün_puan=(puan5+puan4+puan3+puan2+puan1)/100.0
        gt2=soup2.find("div",attrs={"class":"right padding"}).find("span").get_text().strip().upper()
        strn=url[len("https://www.vatanbilgisayar.com/"):]
        sy=strn.find("-")
        ürün_marka=strn[:sy].upper().strip()
        if ürün_marka.find("MACBOOK")!=-1:
            ürün_marka=ürün_marka.replace("MACBOOK","APPLE")
        print(ürün_marka)
        global model
        gt3=soup2.find("div",attrs={"class":"clearfix prod-code-rank"})
        model=gt3.find("div",attrs={"class":"product-list__product-code pull-left product-id"}).get("data-productcode")
        print(model)
        açiklamalari_cek(model,soup2,ürün_marka,url,ürün_fiyat,ürün_adi,ürün_puan,ürün_resim)               
    except:
        pass



site_url="https://www.vatanbilgisayar.com/notebook/"
for k in range(2,20):
        istek=requests.get(site_url,headers=header,verify=False)
        soup=BeautifulSoup(istek.content,"lxml")
        gt1=soup.find_all("a",attrs={"class":"product-list__image-safe-link sld"})
        ürün_url=[]
        for i in gt1:
            ürün_url.append("https://www.vatanbilgisayar.com"+i.get("href"))
        for j in ürün_url:
            ürün_bilgileri_bul(j)
        site_url="https://www.vatanbilgisayar.com/notebook/?page="+str(k)

  


ACER
A315-23-R0U2
ACER ACER ASPIRE 3 3.NESIL RYZEN 5 3500U-8GB-256GB SSD-15.6INC-FREEDOS A315-23-R0U2 FREEDOS AMD RYZEN™ 5 RYZEN 3.NESIL 8 GB 256 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/acer/thumb/134837-1-2_large.jpg 7.999 https://www.vatanbilgisayar.com/acer-aspire-3-3-nesil-ryzen-5-3500u-8gb-256gb-ssd-15-6inc-freedos.html
LENOVO
81VU00FQTX
LENOVO LENOVO IDEAPAD 1 CELERON N4020-4GB-128GB SSD-14INC-W11 81VU00FQTX WINDOWS INTEL CELERON BELIRTILMEMIŞ 4 GB 128 GB SSD 14  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/lenovo/thumb/132731-1-1_large.jpg 4.999 https://www.vatanbilgisayar.com/lenovo-ideapad-1-celeron-n4020-4gb-128gb-ssd-14-w11.html
LENOVO
82H80252TX
LENOVO LENOVO IDEAPAD 3 11.NESIL CORE I3 1115G4-8GB-256GB SSD-15.6INC-W11 82H80252TX WINDOWS INTEL CORE I3 11.NESIL 8 GB 256 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/lenovo/thumb/132732-1_large.jpg 10.999 https://www.vatanbilgisayar.com/lenovo-ideapad-3-11-nesil-core-i3-1115g4-8gb

MBA
Z15SM2JT16512-
MBA MACBOOK AIR Z15SM2JT16512 M2 16GB-512GB SSD-LIQUID RETINA-13.6INC-UZAY GRISI Z15SM2JT16512- MACOS M2 BELIRTILMEMIŞ 16 GB 512 GB SSD 13.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/apple/thumb/135114-1_large.jpg 33.999 https://www.vatanbilgisayar.com/mba-13inc-ug-m2-16gb-512gbssd-ozel-siparis.html
CASPER
G770.1140-8VL0T-B
CASPER CASPER EXCALIBUR 11.NESIL CORE I5 11400H-8GB-500GB SSD-15.6INC-RTX3050TI 4GB-W11 G770.1140-8VL0T-B WINDOWS INTEL CORE I5 11.NESIL 8 GB 500 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/casper/thumb/134854-1_large.jpg 18.999 https://www.vatanbilgisayar.com/casper-excalibur-11-nesil-core-i5-11400h-8gb-500gb-ssd-15-6inc-rtx3050ti-4gb-w11.html
HONOR
5301ABDG
HONOR HONOR MAGICBOOK 16 5.NESIL RYZEN 5 5600H-16GB-512GB SSD-16.1INC-W11 5301ABDG WINDOWS AMD RYZEN™ 5 RYZEN 5.NESIL 16 GB 512 GB SSD 16.1  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/honor/thumb/132238-1_large.jpg 15.999 https://www.vatanbilgisayar.com/h

ASUS
K513EP-L1745W
ASUS ASUS VIVOBOOK 15 OLED 11.NESIL CORE I5 1135G7-8GB-512GB SSD-15.6-MX330 2GB-W11 K513EP-L1745W WINDOWS INTEL CORE I5 11.NESIL 8 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/1-133478_large.jpg 20.499 https://www.vatanbilgisayar.com/asus-vivobook-15-oled-11-nesil-core-i5-1135g7-8gb-512gb-ssd-15-6-mx330-2gb-w11.html
ASUS
FX506LHB-HN323W-Gaming
ASUS ASUS TUF GAMING 15 10.NESIL CORE I5 10300H-8GB-512GB SSD-15.6INC-GTX1650 4GB-W11 FX506LHB-HN323W-Gaming WINDOWS INTEL CORE I5 10.NESIL 8 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/133063-1_large.jpg 19.999 https://www.vatanbilgisayar.com/asus-tuf-gaming-15-10-nesil-core-i5-10300h-8gb-512gb-ssd-15-6inc-gtx1650-4gb-w10.html
MSI
CREATOR Z17 A12UGST-047TR
MSI MSI CREATOR Z17 12.NESIL CORE I7 12700H-32GB-1TB SSD-17INC-RTX3070TI 8GB-W11 PRO CREATOR Z17 A12UGST-047TR WINDOWS INTEL CORE I7 12.NESIL 32 GB 1 TB SSD 17  0.0 https://cdn.vatanbilgisayar.com/Upl

CASPER
F500.1135-8E00T-G-F
CASPER CASPER NIRVANA 11.NESIL CORE I5 1135G7-8GB-500GB SSD-15.6INC-W11 F500.1135-8E00T-G-F WINDOWS INTEL CORE I5 11.NESIL 8 GB 500 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/casper/thumb/1-133582_large.jpg 12.999 https://www.vatanbilgisayar.com/casper-nirvana-11-nesil-core-i5-1135g7-8gb-500gb-ssd-15-6inc-w11.html
CASPER
F500.1115-8D00T-G-F
CASPER CASPER NIRVANA 11.NESIL CORE I3 1115G4-8GB-240GB SSD-15.6INC-W11 F500.1115-8D00T-G-F WINDOWS INTEL CORE I3 11.NESIL 8 GB 240 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/casper/thumb/1-133581_large.jpg 9.999 https://www.vatanbilgisayar.com/casper-nirvana-11-nesil-core-i3-1115g4-4gb-120gb-ssd-15-6inc-w11.html
MSI
PULSE GL76 12UEK-248TR
MSI MSI PULSE GL76 12.NESIL CORE I7 12700H-32GB-1TB SSD-17.3INC-RTX3060 6GB-W10 PULSE GL76 12UEK-248TR WINDOWS INTEL CORE I7 12.NESIL 32 GB 1 TB SSD 17.3  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/msi/thumb/1-133326_large.jpg 51.805 http

ASUS
FX506HM-HN114W-Gaming
ASUS ASUS TUF GAMING F15 11.NESIL CORE I5 11400H-16GB-512GB SSD-15.6-RTX3060 6GB-W11 FX506HM-HN114W-Gaming WINDOWS INTEL CORE I5 11. NESIL 16 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/135784-1_large.jpg 29.999 https://www.vatanbilgisayar.com/asus-tuf-gaming-f15-11-nesil-core-i5-11400h-16gb-512gb-ssd-15-6-rtx3060-6gb-w11.html
ASUS
K6500ZH-MA007W
ASUS ASUS VIVOBOOK PRO 15 OLED 12.NESIL CORE I5 12500H-16GB-512SSD-15-GTX1650 4GB-W11 K6500ZH-MA007W WINDOWS INTEL CORE I5 12. NESIL 16 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/135782-1_large.jpg 35.621 https://www.vatanbilgisayar.com/asus-vivobook-pro-15-oled-11-nesil-core-i5-12500h-16gb-512ssd-14-gtx1650-4gb-w11.html
CASPER
C500.1135-8V00T-G-F
CASPER CASPER NIRVANA 11.NESIL CORE I5 1135G7-8GB-500GB SSD-15.6INC-W11 C500.1135-8V00T-G-F WINDOWS INTEL CORE I5 11. NESIL 8 GB 500 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/

...

ASUS
X515JA-BQ2070W
ASUS ASUS X515JA 10.NESIL CORE I5 1035G1-8GB-512GB SSD-15.6INC-W11 X515JA-BQ2070W WINDOWS INTEL CORE I5 10.NESIL 8 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/133033-1_large.jpg 13.499 https://www.vatanbilgisayar.com/asus-x515ja-10-nesil-core-i5-1035g1-8gb-512gb-ssd-15-6inc-w11.html
HUAWEI
53012XHK
HUAWEI HUAWEI MATEBOOK 16 5.NESIL RYZEN 7 5800H-16GB-512GB SSD-16INC-W11 53012XHK WINDOWS AMD RYZEN™ 7 RYZEN 5.NESIL 16 GB 512 GB SSD 16  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/huawei/thumb/133008-1_large.jpg 21.999 https://www.vatanbilgisayar.com/huawei-matebook-16-5-nesil-ryzen-7-5800h-16gb-512gb-ssd-16inc-w11.html
MSI
SUMMIT E16 FLIP A12UDT-019TR
MSI MSI SUMMIT E16 FLIP 12.NESIL CORE I7 1280P-32GB-2TB SSD-16-RTX3050TI 4GB-W11PRO SUMMIT E16 FLIP A12UDT-019TR WIN10PRO INTEL CORE I7 12.NESIL 32 GB 2 TB SSD 16  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/msi/thumb/132978-1_large.jpg 62.867 https://www.vatanbilgisayar

MSI
BRAVO 15 B5DD-210TR
MSI MSI BRAVO 15 5.NESIL RYZEN 7 5800H-16GB-512GB SSD-15.6INC-RX5500M 4GB-W11 BRAVO 15 B5DD-210TR WINDOWS AMD RYZEN™ 7 RYZEN 5.NESIL 16 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/msi/thumb/132094-1_large.jpg 25.295 https://www.vatanbilgisayar.com/msi-bravo-15-5-nesil-ryzen-7-5800h-16gb-512gb-ssd-15-6-rx5500m-4gb-w11.html
MSI
RAIDER GE76 12UGS-025TR
MSI MSI GE76 RAIDER 12.NESIL CORE I7 12700H-32GB-1TB SSD-17.3INC-RTX3070TI 8GB-W11 RAIDER GE76 12UGS-025TR WINDOWS INTEL CORE I7 12.NESIL 32 GB 1 TB SSD 17.3  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/msi/thumb/132091-1_large.jpg 65.080 https://www.vatanbilgisayar.com/msi-ge76-raider-12-nesil-core-i7-12700h-32gb-1tb-ssd-17-3-rtx3070ti-8gb-w11.html
ASUS
GU603HE-KR013T-Gaming
ASUS ASUS ROG ZEPHYRUS M16 11.NESIL CORE I7 11800H-16GB-512GBSSD-16-RTX3050TI 4GB-W10 GU603HE-KR013T-Gaming WINDOWS INTEL CORE I7 11.NESIL 16 GB 512 GB SSD 16  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/

ASUS
X515EP-BQ259T
ASUS ASUS X515EP CORE I7 1165G7-16GB-512GB SSD-15.6INC-MX330 2GB-W10 X515EP-BQ259T WINDOWS INTEL CORE I7 11.NESIL 16 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/130727-1_large.jpg 24.999 https://www.vatanbilgisayar.com/asus-x515ep-core-i7-1165g7-2-8ghz-16gb-ram-512gb-ssd-15-6-mx330-2gb-w10.html
ASUS
K3400PH-KM018T
ASUS ASUS VIVOBOOK PRO 14 OLED 11.NESIL CORE I5 11300H-16GB-512SSD-14-GTX1650 4GB-W10 K3400PH-KM018T WINDOWS INTEL CORE I5 11.NESIL 16 GB 512 GB SSD 14  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/130726-1_large.jpg 27.999 https://www.vatanbilgisayar.com/asus-vivobook-pro-14-oled-corei5-11300h-3-1ghz-16gb-512gbssd-14-gtx1650-4gb-w10.html
ASUS
K3400PH-KM012T
ASUS ASUS VIVOBOOK PRO 14 OLED 11.NESIL CORE I7 11370H-16GB-512SSD-14-GTX1650 4GB-W10 K3400PH-KM012T WINDOWS INTEL CORE I7 11.NESIL 16 GB 512 GB SSD 14  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/130725-1_large.jpg 31.999 https://

MSI
SUMMIT E16 FLIP A11UCT-041TR
MSI MSI SUMMIT E16 FLIP 11.NESIL CORE I7 1195G7-32GB-1TB SSD-16-RTX3050 4GB-W10PRO SUMMIT E16 FLIP A11UCT-041TR WIN10PRO INTEL CORE I7 11.NESIL 32 GB 1 TB SSD 16  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/msi/thumb/129777-1_large.jpg 45.113 https://www.vatanbilgisayar.com/msi-summit-e16-flip-core-i7-1195g7-2-9ghz-32gb-1tb-ssd-16-rtx3050-4gb-w10-pro.html
HP
4H0T9EA
HP HP ENVY 13-BA1002NT EVO 11.NESIL CORE I5 1135G7-8GB-512GB SSD-13.3INC-W10 4H0T9EA WINDOWS INTEL CORE I5 11.NESIL 8 GB 512 GB SSD 13.3  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/hp/thumb/129026_large.jpg 11.999 https://www.vatanbilgisayar.com/hp-envy-13-ba1002nt-core-i5-1135g7-2-4ghz-8gb-512gb-ssd-13-3-int-w10.html
HP
4H0W6EA
HP HP PAVILION 15-EH1007NT 5.NESIL RYZEN 7 5700U-8GB-512GB SSD-15.6INC-W10 4H0W6EA WINDOWS AMD RYZEN™ 7 RYZEN 5.NESIL 8 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/hp/thumb/129024_large.jpg 11.499 https://www.vatanbilgisayar.

ASUS
G713QR-K4093T-Gaming
ASUS ASUS ROG STRIX G17 5.NESIL RYZEN 9 5900HX-32GB-1TB SSD-17.3INC-RTX3070 8GB-W10 G713QR-K4093T-Gaming WINDOWS AMD RYZEN™ 9 RYZEN 5.NESIL 32 GB 1 TB SSD 17.3  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/121946_large.jpg 67.999 https://www.vatanbilgisayar.com/asus-rog-strix-g17-ryzen-9-5900hx-3-3ghz-32gb-1tbssd-17-3-rtx3070-8gb-w10.html
LENOVO
82HS0069TX
LENOVO LENOVO IDEAPAD FLEX 5 11.NESIL CORE I5 1135G7-16GB-512GB SSD-14INC-MX450 2GB-W10 82HS0069TX WINDOWS INTEL CORE I5 11.NESIL 16 GB 512 GB SSD 14  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/lenovo/thumb/121900_large.jpg 19.999 https://www.vatanbilgisayar.com/lenovo-ideapad-flex-5-core-i5-1135g7-2-4ghz-16gb-512gb-ssd-14-mx450-2gb-w10.html
ASUS
X515JP-BQ119T
ASUS ASUS X515JF 10.NESIL CORE I7 1065G7-8GB-512GB SSD-15.6INC-MX330 2GB-W10 X515JP-BQ119T WINDOWS INTEL CORE I7 10.NESIL 8 GB 512 GB SSD 15.6  0.0 https://cdn.vatanbilgisayar.com/Upload/PRODUCT/asus/thumb/121791_large.jpg 19.9

In [5]:
import pypyodbc 
import datetime
import time
db = pypyodbc.connect(
    'Driver={SQL Server};'
    'Server=.;'
    'Database=UrunlerDeneme;'
    'Trusted_Connection=True;'
)
sqlimlec=db.cursor()
global siteId
def sonMarkaId():
    kontrol=sqlimlec.execute('SELECT * FROM tbl_markalar ORDER BY MarkaId DESC')
    tablo = sqlimlec.fetchone()
    return tablo[0]
def marka_ekle(markaAd):
    sqlimlec.execute('SELECT * FROM tbl_markalar')
    tablo = sqlimlec.fetchall()
    if tablo:
        for i in range(0,len(tablo)):
            if tablo[i][1].find(markaAd)!=-1:
                return tablo[i][0]
    veriler=[]
    komut = 'INSERT INTO tbl_markalar VALUES(?)'
    veriler.append(markaAd)
    sonuc = sqlimlec.execute(komut,veriler)
    db.commit()
    markaId=sonMarkaId()
    return markaId
def model_id():
    kontrol=sqlimlec.execute('SELECT * FROM tbl_modeller ORDER BY ModelId DESC')
    tablo = sqlimlec.fetchone()
    return tablo[0]
def model_ekle(markaId,modelNo,modelResim,modelPuan):
    komut = 'INSERT INTO tbl_modeller VALUES(?,?,?,?)'
    veriler=(modelNo,markaId,modelResim,modelPuan)
    sonuc = sqlimlec.execute(komut,veriler)
    db.commit()
    mdl=model_id()
    return mdl

def model_kontrol(modelNo):
    sqlimlec.execute('SELECT * FROM tbl_modeller')
    tablo = sqlimlec.fetchall()
    kontrol1=modelNo.upper().replace(" ","")
    if tablo:
        for i in tablo:
            kontrol2=i[1].upper().replace(" ","")
            if kontrol2.find(kontrol1)!=-1 or kontrol1.find(kontrol2)!=-1:
                return i[0]
        
    return -1
def urun_ekle(ürünAd,modelId,fiyat,url):
                komut1 = 'INSERT INTO tbl_urunler VALUES(?,?,?,?,?)'
                tarih =  datetime.datetime.now()
                veriler=(ürünAd,modelId,url,fiyat,tarih)
                sqlimlec.execute(komut1,veriler)
                db.commit()
                urn=sonurunId_bul()
                return urn
def sonurunId_bul():
    sqlimlec.execute('SELECT * FROM tbl_urunler ORDER BY UrunId DESC')
    tablo = sqlimlec.fetchone()
    return tablo[0]
def urun_aciklama_ekle(urunId,markaId,isletimSistemi,islemciTip,islemciNesli,ram,diskBoyut,diskTur,ekran_boyutu):
    komut = 'INSERT INTO tbl_urunAciklama VALUES(?,?,?,?,?,?,?,?,?,?)'
    veriler=(urunId,markaId,isletimSistemi,islemciTip,islemciNesli,ram,diskBoyut,diskTur,ekran_boyutu,siteId)
    sqlimlec.execute(komut,veriler)
    db.commit()
def urun_guncelle(fiyat,Url,UrunId):
               sonuc =sqlimlec.execute('UPDATE tbl_urunler SET Fiyat = ? ,UrunUrl= ? WHERE UrunId = ?',(fiyat,Url,UrunId))
               db.commit()
def ayni_urun_varmi(islemciTip,islemciNesli,ram,diskBoyut,diskTur,isletimSistemi,ekran_boyutu):
    komut='SELECT * FROM tbl_urunAciklama  Where isletimSistemi=? and islemciNesli=? and ram=? and diskBoyut=? and diskTur=? and islemciTip=? and ekranBoyut=? and siteId=?'
    veriler=(isletimSistemi,islemciNesli,ram,diskBoyut,diskTur,islemciTip,ekran_boyutu,siteId)
    sqlimlec.execute(komut,veriler)
    bilgiler=sqlimlec.fetchone()
    if bilgiler:
        return bilgiler[1]
    return -1
def ayni_site_model(modelId):
    komut='SELECT t2.UrunId FROM tbl_urunAciklama t1,tbl_urunler t2 where t1.UrunId=t2.UrunId and t2.modelId=? and t1.siteId=?'
    veriler=(modelId,siteId)
    sqlimlec.execute(komut,veriler)
    bilgiler=sqlimlec.fetchone()
    if bilgiler:
        return bilgiler
    return -1
def tablo_bosmu(tabloAd):
    sqlimlec.execute('SELECT * FROM '+tabloAd)
    tablo = sqlimlec.fetchall()
    return tablo
def sontSiteId_bul():
    sqlimlec.execute('SELECT * FROM tbl_siteler ORDER BY SiteId DESC')
    tablo = sqlimlec.fetchone()
    return tablo[0]
def site_ekle(siteIsmi,siteLogo):
    sqlimlec.execute('SELECT * FROM tbl_siteler')
    tablo = sqlimlec.fetchall()
    if tablo:
        for i in range(0,len(tablo)):
            if tablo[i][1].find(siteIsmi)!=-1:
                return tablo[i][0]
    veriler=[]
    komut = 'INSERT INTO tbl_siteler VALUES(?,?)'
    veriler=(siteIsmi,siteLogo)
    sonuc = sqlimlec.execute(komut,veriler)
    db.commit()
    site=sontSiteId_bul()
    return site


siteId=site_ekle("Vatan","https://www.vatanbilgisayar.com/assets/dist/images/vatanlogo.svg")
print(len(ürün_bilgileri))
for x in ürün_bilgileri:
    ürünMarka=x[0]
    ürünAd=x[1]
    ModelNo=x[2]
    isletimSistemi=x[3]
    islemciTip=x[4]
    islemciNesli=x[5]
    ram=x[6]
    diskBoyut=x[7]
    diskTur=x[8]
    ekranBoyutu=x[9]
    ürünPuan=x[10]
    ürünResim=x[11]
    fiyat=float(x[12].replace(".","").replace(",","."))
    url=x[13]
    kontrol=-1
    markaId=marka_ekle(ürünMarka)
    kontrol=model_kontrol(ModelNo)
    if kontrol!=-1:
        modelId=kontrol
        bilgiler=ayni_urun_varmi(islemciTip,islemciNesli,ram,diskBoyut,diskTur,isletimSistemi,ekranBoyutu)
        if bilgiler!=-1:
            UrunId=bilgiler
            urun_guncelle(fiyat,url, UrunId)
        else:
            ayniId=ayni_site_model(modelId)
            if ayniId==-1:
                UrunId=urun_ekle(ürünAd,modelId,fiyat,url)
                urun_aciklama_ekle(UrunId,markaId,isletimSistemi,islemciTip,islemciNesli,ram,diskBoyut,diskTur,ekranBoyutu)
            else:
                UrunId=bilgiler
                urun_guncelle(fiyat,url, UrunId)
    else:
        modelId=model_ekle(markaId,ModelNo,ürünResim,ürünPuan)
        UrunId=urun_ekle(ürünAd,modelId,fiyat,url)
        urun_aciklama_ekle(UrunId,markaId,isletimSistemi,islemciTip,islemciNesli,ram,diskBoyut,diskTur,ekranBoyutu)

 
 
db.close()

260


In [3]:
pd.DataFrame(ürün_bilgileri)
df=pd.DataFrame(ürün_bilgileri)
df.to_excel("vatanDatabase2.xlsx")